In [1]:
import torch
from functions import get_loader, get_model

device = 'cuda' if torch.cuda.is_available() else 'cpu'
_, _, loader = get_loader()
model, _, _ = get_model()

model.classifier

Linear(in_features=768, out_features=10, bias=True)

In [2]:
from peft import LoraConfig, TaskType, get_peft_model, LoftQConfig

#此处不再指定task_type
config = LoraConfig(inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)

model = get_peft_model(model, config)

model.print_trainable_parameters()

model.classifier

trainable params: 786,432 || all params: 252,033,802 || trainable%: 0.31203433577532586


Linear(in_features=768, out_features=10, bias=True)

In [3]:
import datetime

#正常训练
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
model.to(device)

now = datetime.datetime.now()
for i, data in enumerate(loader):
    for k, v in data.items():
        data[k] = v.to(device)
    out = model(**data)
    out.loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    optimizer.zero_grad()

    if i % 1 == 0:
        labels = data['labels']
        logits = out['logits'].argmax(1)
        acc = (labels == logits).sum().item() / len(labels)

        print(i, len(loader), out.loss.item(), acc)

datetime.datetime.now() - now

0 62 2.2774503231048584 0.125
1 62 2.2287027835845947 0.1875
2 62 2.3163657188415527 0.15625
3 62 2.209228992462158 0.21875
4 62 2.1856229305267334 0.15625
5 62 2.2188992500305176 0.1875
6 62 2.0847771167755127 0.34375
7 62 2.1538450717926025 0.3125
8 62 2.0640904903411865 0.375
9 62 1.9859848022460938 0.5625
10 62 1.953343152999878 0.59375
11 62 1.9733211994171143 0.6875
12 62 1.9323341846466064 0.6875
13 62 1.8512815237045288 0.78125
14 62 1.9471282958984375 0.625
15 62 1.7356795072555542 0.90625
16 62 1.759588599205017 0.875
17 62 1.6408013105392456 0.96875
18 62 1.7141221761703491 0.8125
19 62 1.6078336238861084 0.9375
20 62 1.5887126922607422 1.0
21 62 1.4986778497695923 0.96875
22 62 1.5481253862380981 0.875
23 62 1.4178872108459473 1.0
24 62 1.371886968612671 1.0
25 62 1.4725046157836914 0.9375
26 62 1.3218789100646973 0.96875
27 62 1.359554409980774 1.0
28 62 1.2736034393310547 0.96875
29 62 1.2936562299728394 0.96875
30 62 1.1820000410079956 1.0
31 62 1.1500431299209595 1.0
32

datetime.timedelta(seconds=25, microseconds=576708)